# Testing

This chapter is about automatic unit testing of Python code. For this purpose, the *pytest* library is used.

An alternative would be the *unittest* bulit-in library of Python, but *pytest* is preferred due to its easier syntax.

Unit Tests are a must-have for any code which is used for productive purposes. They greatly enhance code quality and the possibility to refactor or migrate the code to new Python/ library versions.

In [2]:
!conda install pytest --yes # this package is not included in Scipy Notebook

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytest


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    atomicwrites-1.3.0         |             py_0           9 KB  conda-forge
    importlib_metadata-0.18    |           py37_0          36 KB  conda-forge
    more-itertools-4.3.0       |        py37_1000          83 KB  conda-forge
    pluggy-0.12.0              |             py_0          18 KB  conda-forge
    py-1.8.0                   |             py_0          65 KB  conda-forge
    pytest-5.0.0               |           py37_0         348 KB  conda-forge
    zipp-0.5.1                 |             py_0           7 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         566 KB

The following NEW packages will be INSTALLED:

  atomicw

## Starting Tests

In [2]:
!pytest ../tests 

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.0.0, py-1.8.0, pluggy-0.12.0
rootdir: /home/jovyan/Tutorial
collected 8 items                                                              

../tests/test_cases.py .F......                                          [100%]

=================================== FAILURES ===================================
_______________________________ test_which_fails _______________________________

    def test_which_fails():
        """test case which always fails
        - just for illustration here, do not put something like this in real test cases!"""
>       assert my_module.double_me(1) == 3
E       assert 2 == 3
E        +  where 2 = <function double_me at 0x7feda60ed6a8>(1)
E        +    where <function double_me at 0x7feda60ed6a8> = my_module.double_me

../tests/test_cases.py:15: AssertionError
====================== 1 failed, 7 passed in 0.33 seconds ======================

Pytest gives a summary of the number of succeeded and failed test cases. For the latter, a detailed breakdown of the test failure is given.

## Setup of Test Cases

### Directory Structure

It is recommended to put the application code and tests into different directories.

In the following, a directory structure like below is assumed:

    app_route/
        code/
            my_module.py
            ...
        tests/
            conftest.py
            fixtures.py
            utils.py
            test_cases.py
            ...
            
*code* is the base directory for all application code to be tested.

### Test Files

In [1]:
%cat ../tests/conftest.py

"""
This module is always executed automatically when pytest is started
"""

# set Pythonpath to include modue code
import sys
import os
new_paths = [os.path.abspath('../code'), os.path.abspath('../tests'),]
for new_path in new_paths:
    if new_path not in sys.path:
        sys.path.append(new_path)

# import fixtures
from fixtures import * # one of the very few cases where import * is OK


The file *conftest.py* is always executed automatically when pytest is started.
It is used here to add the application code directory to the Pythonpath (if it is not already there) and to import the fixtures (for definition of fixtures see below).

Note that it is possible (and often done) to define fixtures and directly in *conftest.py*, but due to scalability and readability I recommend to use different modules for them and only import them in *conftest.py*.

In [3]:
%cat ../tests/test_cases.py

import pytest
import my_module
import utils

def test_double_1():
    """simple test case"""
    assert my_module.double_me(1) == 2 # asserts are used in pytest to check expectations
    
    # you can put multiple asserts into one test function, but it is still considered as a single test case
    assert my_module.double_me(-5) == -10 

def test_which_fails():
    """test case which always fails 
    - just for illustration here, do not put something like this in real test cases!"""
    assert my_module.double_me(1) == 3

def test_error_message():
    """checks raising of error"""
    with pytest.raises(TypeError):
        assert my_module.double_me({1, 2, 3})

@pytest.mark.parametrize('input, expected', [(2, 4), (3, 6), (4, 8)])
def test_double_2(input, expected):
    """testing multiple inputs with corresponding expected outputs"""
    assert my_module.double_me(input) == expected
    
def test_double_3(vals_for_test):
    """using pytest.fixture for defining test values and expecte

In this file the test cases are defined.

Each test case is a function whose name starts with *test_*.
As funtion arguments, fixtures or parameters can be used.

Test cases can be defined in more than one .py file. The name of the .py files is not relevant, but it is required that they are located in the test directory.

In [31]:
%cat ../tests/fixtures.py

import pytest
import my_module

import numpy as np

@pytest.fixture(scope='session') # this fixture is used for the whole test session
def multiplier7():
    return my_module.Multiplier(7)

@pytest.fixture # default is to create a fixture object for every test function
def vals_for_test():
    vals = np.random.randn(20)
    expectations = vals*2
    return vals, expectations


Fixtures are definitions of Python objects which can be used in any test case.

Use this for objects which are required by multiple test cases. If the setup of the fixture takes a long time, set the *scope* parameter to e.g. *session* to use the same fixture object for multiple test cases (standard is to create a new instance for each test function).

In [6]:
%cat ../tests/utils.py

"""In this module helper functions, etc. for testing are defined."""

def compare_np_arrays(x, y):
    """checks if 2 numpy arrays are identical"""
    return (x == y).all()
    

In this module helper functions, etc. for testing are defined.
Functions which are required by multiple test cases, potentially located in different .py files, should be extracted into a helper module.